In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, WeekdayLocator,\
    DayLocator, MONDAY,date2num,num2date,AutoDateLocator
from matplotlib.finance import quotes_historical_yahoo_ohlc, candlestick_ohlc,candlestick2_ochl,volume_overlay3

from stockapp import models as stkmd
from dataapp import models as dtamd
from dataapp import tasks as dtatks
from dataapp import libs as dtalibs
from featureapp import libs as ftlibs
from featureapp import models as ftmd
from stockapp import tasks as stktks
from stockapp import libs as stklibs
import featureapp.models as ftmd
import featureapp.tasks as fttks
import queryapp.models as qrymd
import queryapp.tasks as qrytks

import charts.chartservers.libs as chservlibs
import charts.libs as chlibs

from datascience import models as dtscmd
from datascience import tasks as dtsctks
from datascience import libs as dtsclibs


import featureapp as ftapp
import utility as uty
from utility import models as utymd
import itertools as itt
import multiprocessing as mp
from django.db import connection,connections
from django.db import reset_queries
import time 
import pandas as pd
import numpy as np
import inspect
import imp
import datetime
from talib.abstract import *
import utility.models as utmd
import stockapp.libs as stklib
from utility import codemanager as cdmng
from utility import maintenance as mnt
import os 
import json
from django.contrib.auth.models import AnonymousUser
import threading

stk=stkmd.Stockmeta.objects.get(Symbol='TSLA')
Fromdate=pd.datetime(2008,1,1)
Todate=pd.datetime.today()
Trange=pd.date_range(Fromdate,Todate)
Trange=[T.date() for T in Trange if T.weekday()<=4]

import json
# fttks.computefeatuers(stk.id,Trange)

# Charting

In [ ]:
entries=[
    {'Symbol':'TSLA','TF':pd.datetime(2012,1,1).date(),'T0':pd.datetime(2011,1,1).date()  },
    {'Symbol':'AAPL','TF':pd.datetime(2012,1,1).date(),'T0':pd.datetime(2011,1,1).date()  }
]
chservlibs.request_db_charts(entries,5003)
# img=chlibs.CurrentByFutureChart_bydb(entries[0]['T0'],entries[0]['TF'],entries[0]['Symbol'],indicatorlist=(),pricecols=(),querycols=(),featcols=())


# Running Features

In [ ]:
print "---------------- features-------------------"
featurecodes=ftmd.FeatureComputeCode.objects.all()
computecode=featurecodes[0]
computeclass=computecode.importcomputeclass()
CF=computeclass(stk.id,Trange)
CF.computeall(skipdone=True)
# CF.saveall()
print CF.getfeaturelist()

CF.df=CF.addindicators(CF.df,[
        {'name':'SMAstd','timeperiod':20,'colname':'SMAstd20'},
        {'name':'EMAstd','timeperiod':8,'colname':'EMAstd8'},
    ])

# Running Queries

In [ ]:
querycodes=qrymd.QueryComputeCode.objects.all()
computecode=querycodes[0]
computeclass=computecode.importcomputeclass()
CQ=computeclass(stk.id,Trange)
CQ.computeall(skipdone=True)
CQ.saveall()

In [ ]:
CQ.OutcomeCharts()

In [ ]:
CQ.getquerylist()

In [ ]:
CQ.chartfeatures(addpricecols=(),ip=5562,
addfeatcols=[
    ['CCI5','CCI50'],
    ['PastPROFIT10days','PastLOSS10days'],['FutPROFIT10days','FutLOSS10days']
],
addquerycols=[
    'CCICHERRIES',
]
)

# Creating Dataset

In [ ]:
from datascience import libs as dtsclibs

data_id=dtsclibs.register_dataset("PredictReturn",{'description': "Testing the algorithms on TSLA to predict next 10 day return \n"+
                                 "Data taken on every Friday"},'RawProcessed',"OnlyTSLA","1",'npz','Regression',
                                    use_project_ifexists=True)

# Registering function

In [ ]:
from datascience import libs as dtsclibs
import pandas as pd

@dtsclibs.register_compfunc(RequiredImports=['import pandas as pd','from dataapp import libs as dtalibs'],overwrite_if_exists=False)
def extractdataset(data_id,Symbol):
    """
    @funcName : test
    @input x : an int
    @output df : pd.DataFrame, some random 2 by 2 
    @description : takes an int and then returns a dummy dataframe. This is just for testing purposes
    @Source : 
    def extractdataset(data_id,Symbol):
        window=60
        window_fut=30
        Tfs=map(lambda x: ( (x.date()-pd.Dateoffset(window)).date(),x.date(), (x.date()+pd.Dateoffset(window_fut)).date() ),
                pd.date_range(start=pd.datetime(2010,1,1),end=pd.datetime.today(),freq='W-MON') )

        N=len(Tfs)
        dfinstants=pd.DataFrame({'T0':map(lambda x: x[0],Tfs),'TF':map(lambda x: x[1],Tfs),'Symbol':[Symbol]*N})
        X,X_Meta=dtalibs.Getbatchdata(dfinstants)

        dfinstants=pd.DataFrame({'T0':map(lambda x: x[1],Tfs),'TF':map(lambda x: x[2],Tfs),'Symbol':[Symbol]*N})
        Y,Y_Meta=dtalibs.Getbatchdata(dfinstants)

        shard=dtscmd.DataShard(Data__id=data_id)
        shard.Info['X_Meta']=X_Meta
        shard.Info['Y_Meta']=Y_Meta
        shard.save()

        np.savez_compressed(shard.shardpath(),X=X,Y=Y)
    """
    
    window=60
    window_fut=30
    Tfs=map(lambda x: ( (x.date()-pd.Dateoffset(window)).date(),x.date(), (x.date()+pd.Dateoffset(window_fut)).date() ),
            pd.date_range(start=pd.datetime(2010,1,1),end=pd.datetime.today(),freq='W-MON') )
    
    N=len(Tfs)
    dfinstants=pd.DataFrame({'T0':map(lambda x: x[0],Tfs),'TF':map(lambda x: x[1],Tfs),'Symbol':[Symbol]*N})
    X,X_Meta=dtalibs.Getbatchdata(dfinstants)
    
    dfinstants=pd.DataFrame({'T0':map(lambda x: x[1],Tfs),'TF':map(lambda x: x[2],Tfs),'Symbol':[Symbol]*N})
    Y,Y_Meta=dtalibs.Getbatchdata(dfinstants)
    
    shard=dtscmd.DataShard(Data__id=data_id)
    shard.Info['X_Meta']=X_Meta
    shard.Info['Y_Meta']=Y_Meta
    shard.save()
    
    np.savez_compressed(shard.shardpath(),X=X,Y=Y)

extractdataset.id


# Creating initial Stock price dataset

In [ ]:
from datascience import models as dtscmd
from datascience import tasks as dtsctks
from datascience import libs as dtsclibs

projid,dataid=dtsclibs.register_dataset(project_Name="PredictReturn",project_Info={'description': "Testing the algorithms on TSLA to predict next 10 day return \n"+
                                 "Data taken on every Friday"},
                                        Datatype='RawProcessed',GroupName="AllStocks",tag="1",
                                        data_format='npz',Modeltype='Regression',
                                        TransformedFromDataId=None,TransFuncId=None, use_project_ifexists=True)

# dtsctks.CreateStockData_2(360,60,dataid,None)

In [ ]:
print dtscmd.Data.objects.filter(Project__id=7,GroupName="AllStocks",tag="1",Datatype='RawProcessed',
                           Dataformat='npz',Modeltype='Regression')
data = dtscmd.Data.objects.get(Project__id=7,GroupName="AllStocks",tag="1",Datatype='RawProcessed',
                           Dataformat='npz',Modeltype='Regression')

data.datapath()

In [ ]:
dtscmd.DataShard.objects.filter(Data=data).delete()

In [ ]:
dtscmd.ComputeFunc.objects.all()

# Fonsuncti on shards

In [1]:
from datascience import models as dtscmd
from datascience import tasks as dtsctks
from datascience import libs as dtsclibs
from celery import group
import time
import pickle as pkl

@dtsclibs.register_compfunc(Group='test',overwrite_if_exists=True)
def GetMeta(shardId):
    print "----------------------"
    print shardId
    print "----------------------"
    
    shard=dtscmd.DataShard.objects.get(id=shardId)
    X,Y,Meta=shard.getdata()
    return Meta

M=dtsctks.applyfunc2data(GetMeta.id,6,wait=True)


Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


over writing previous function
saving function :  GetMeta
function id =  19
302 0 304
301 0 304
300 0 304
299 0 304
298 0 304
297 0 304
296 0 304
295 0 304
294 0 304
293 0 304
292 0 304
291 0 304
290 0 304
289 0 304
288 0 304
287 0 304
286 0 304
285 0 304
284 0 304
283 0 304
282 0 304
281 0 304
280 0 304
279 0 304
278 0 304
277 0 304
276 0 304
275 0 304
274 0 304
273 0 304
272 0 304
271 0 304
270 0 304
269 0 304
268 0 304
267 0 304
266 0 304
265 0 304
264 0 304
263 0 304
262 0 304
261 0 304
260 0 304
259 0 304
258 0 304
257 0 304
256 0 304
255 0 304
254 0 304
253 0 304
252 0 304
251 0 304
250 0 304
249 0 304
248 0 304
247 0 304
246 0 304
245 0 304
244 0 304
243 0 304
242 0 304
241 0 304
240 0 304
239 0 304
238 0 304
237 0 304
236 0 304
235 0 304
234 0 304
233 0 304
232 0 304
231 0 304
230 0 304
229 0 304
228 0 304
227 0 304
226 0 304
225 0 304
224 0 304
223 0 304
222 0 304
221 0 304
220 0 304
219 0 304
218 0 304
217 0 304
216 0 304
215 0 304
214 0 304
213 0 304
212 0 304
211 0 304
210 

302 294 304
301 294 304
300 294 304
299 295 304
298 296 304
297 297 304
296 298 304
295 299 304
294 300 304
293 301 304
292 301 304
291 301 304
290 301 304
289 301 304
288 301 304
287 301 304
286 301 304
285 301 304
284 301 304
283 301 304
282 301 304
281 301 304
280 301 304
279 301 304
278 301 304
277 301 304
276 301 304
275 301 304
274 301 304
273 301 304
272 301 304
271 301 304
270 301 304
269 301 304
268 301 304
267 301 304
266 301 304
265 301 304
264 301 304
263 301 304
262 301 304
261 301 304
260 301 304
259 301 304
258 301 304
257 301 304
256 301 304
255 301 304
254 301 304
253 301 304
252 301 304
251 301 304
250 301 304
249 301 304
248 301 304
247 301 304
246 301 304
245 301 304
244 301 304
243 301 304
242 301 304
241 301 304
240 301 304
239 301 304
238 301 304
237 301 304
236 301 304
235 301 304
234 301 304
233 301 304
232 301 304
231 301 304
230 301 304
229 301 304
228 301 304
227 301 304
226 301 304
225 301 304
224 301 304
223 301 304
222 301 304
221 301 304
220 301 304
219 

In [2]:
M

{0: {'MetaX': {'columns': Index([u'Close', u'Open', u'High', u'Low', u'Volume', u'Symbol'], dtype='object'),
   'dfinstants':     Symbol                   T0                   TF  window
   0     KEYW  2009-01-09T00:00:00  2010-01-04T00:00:00     360
   1     KEYW  2009-01-16T00:00:00  2010-01-11T00:00:00     360
   2     KEYW  2009-01-23T00:00:00  2010-01-18T00:00:00     360
   3     KEYW  2009-01-30T00:00:00  2010-01-25T00:00:00     360
   4     KEYW  2009-02-06T00:00:00  2010-02-01T00:00:00     360
   5     KEYW  2009-02-13T00:00:00  2010-02-08T00:00:00     360
   6     KEYW  2009-02-20T00:00:00  2010-02-15T00:00:00     360
   7     KEYW  2009-02-27T00:00:00  2010-02-22T00:00:00     360
   8     KEYW  2009-03-06T00:00:00  2010-03-01T00:00:00     360
   9     KEYW  2009-03-13T00:00:00  2010-03-08T00:00:00     360
   10    KEYW  2009-03-20T00:00:00  2010-03-15T00:00:00     360
   11    KEYW  2009-03-27T00:00:00  2010-03-22T00:00:00     360
   12    KEYW  2009-04-03T00:00:00  2010-03-2

In [ ]:
dtscmd.DataShard.objects.filter(Data__id=6,id__lte=6036).delete()

# Creating derived DataSet

In [2]:
from datascience import models as dtscmd
pr=dtscmd.Project.objects.all()[0]
pr.initialize()

In [3]:
pr.delete()

Signal recieived to delete all data for :  test RawMultiClass  Classification Raw joblib


OSError: [Errno 2] No such file or directory: '/home/venkat/n.adurthi@gmail.com/repos/trade_analytics/trade_analytics/bigdata/datascience/Projects/test/Data/Raw/RawMultiClass_'